In [127]:
import pandas as pd
import networkx as nx
# import streamlit as st
import random
from random import choices
import osmnx as ox
import geopandas as gpd

import folium
from folium import plugins
# from streamlit_folium import folium_static

from shortest_path import get_short_path_graph
from tools import *

In [133]:
# Load DATA:
graph = nx.read_gpickle('./data/graph1.gpickle')
nodes = pd.read_csv('./data/nodes.csv', index_col=0)
edges = pd.read_csv('./data/edges.csv', index_col=0)

In [124]:
# graph = ox.add_edge_speeds(graph)
# graph = ox.add_edge_travel_times(graph)

# ox.save_graph_shapefile(graph, filepath='./data/', encoding='utf-8')

In [125]:
# nx.write_gpickle(graph, "./data/graph1.gpickle")

In [128]:
# nodes = gpd.read_file('./data/nodes.shp')
# edges = gpd.read_file('./data/edges.shp')

In [131]:
# edges.to_csv('./data/edges.csv', index=False)
# nodes.to_csv('./data/nodes.csv', index=False)

In [134]:
nodes

,y,x,street_cou,highway,ref,geometry
osmid,,,,,,
250783680,51.028830,71.463300,2,NaN,NaN,POINT (71.4633004 51.0288297)
250783681,51.068248,71.399443,2,NaN,NaN,POINT (71.3994425 51.0682479)
250783683,51.112727,71.410670,2,NaN,NaN,POINT (71.4106703 51.1127274)
250783684,51.120065,71.412031,3,NaN,NaN,POINT (71.4120309 51.1200654)
250783685,51.136447,71.415708,4,traffic_signals,NaN,POINT (71.4157077 51.1364467)
...,...,...,...,...,...,...
9665797519,51.122354,71.410546,2,NaN,NaN,POINT (71.4105465 51.1223544)
9665953128,51.124823,71.435514,2,NaN,NaN,POINT (71.4355145 51.1248226)
9673353645,51.120703,71.407541,4,NaN,NaN,POINT (71.4075413 51.1207035)


In [117]:
evacuation_points = [8104348645, 2862602003, 8510058624, 1774432269, 8531352025]

In [118]:
# Генерация эфакуаионных пунктов
nodes['is_evacuation'] = 0
nodes.loc[nodes['osmid'].isin(evacuation_points), 'is_evacuation'] = 1

In [103]:
# Simulate loading capacity for heatmap
population = [0, 1, 2]
weights = [0.2, 0.5, 0.3]

# Heatmap will be based on middle points
# of edges:

mid_x = ((edges.geometry.str.replace(
    ')', '').str.replace(
        '(', '').str.replace(
            ',', '').str.split(' ').str[1].astype(float)
 + edges.geometry.str.replace(
    ')', '').str.replace(
        '(', '').str.replace(
            ',', '').str.split(' ').str[3].astype(float)
) 
/ 2)

mid_y = ((edges.geometry.str.replace(
    ')', '').str.replace(
        '(', '').str.replace(
            ',', '').str.split(' ').str[2].astype(float)
 + edges.geometry.str.replace(
    ')', '').str.replace(
        '(', '').str.replace(
            ',', '').str.split(' ').str[4].astype(float)
) 
/ 2)

edges['mid_x'] = mid_x
edges['mid_y'] = mid_y

# HeatMap data according to points
_y = edges['mid_y'].tolist()
_x = edges['mid_x'].tolist()
_heat = [choices(population, weights) for i in range(len(_y))]
heatmap_data = [[_y[i], _x[i], _heat[i][0]] for i in range(len(_y))]

# Assign choices list to workload
edges['workload'] = [elem[0] for elem in _heat]
geometry = edges['geometry'].tolist()
workload = edges['workload'].tolist()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd
import networkx as nx
import osmnx as ox
ox.config(use_cache=True, log_console=False)
import random
import re

def get_short_path_graph(G, nodes, node_a, node_b, ):
    # Input: User location(Start point):
    #        datatype - POINT ShapeFile,
    #        Destination:
    #        datatype - POINT ShapeFile
    # Output: Path:
    #         datatype - List of osmids
    
    node_b = (4959701273 if node_b==None else node_b)

    # Getting shortest path
    route = nx.shortest_path(G, source=node_a, target=node_b, weight="travel_tim_1")
    
    coordinates = []
    for x in route:
        coordinates.append([nodes[nodes['osmid']==x]['y'].to_list()[0], nodes[nodes['osmid']==x]['x'].to_list()[0]])

    return coordinates